##### Named Entity Recognition c/ spaCy

In [1]:
import spacy
from spacy import displacy
from collections import Counter
from pypdf import PdfReader

In [2]:
# usa gpu se tiver
spacy.prefer_gpu()

True

In [3]:
nlp = spacy.load('pt_core_news_sm')
print(nlp.pipe_names)

['tok2vec', 'morphologizer', 'parser', 'lemmatizer', 'attribute_ruler', 'ner']


In [4]:
# processando texto
text = "Maior medalhista da história do Brasil, Rebeca Andrade foi a atleta do país que mais faturou com seu desempenho nas Olimpíadas de Paris."
doc = nlp(text)
print(doc)

Maior medalhista da história do Brasil, Rebeca Andrade foi a atleta do país que mais faturou com seu desempenho nas Olimpíadas de Paris.


In [5]:
template = "{: <10}\t{: <10}\t{}\t{}\t{: <10}\t{}\t{}\t{}\t{: <10}\t{}\t"
header = template.format("TOKEN","LEMMA","POS","TAG","DEP","SHAPE","APHA","STOP","EXPLAIN","MORPH")
print(header)
for token in doc:
    print(template.format(
        token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        token.shape_, token.is_alpha, token.is_stop, spacy.explain(token.tag_), token.morph))

TOKEN     	LEMMA     	POS	TAG	DEP       	SHAPE	APHA	STOP	EXPLAIN   	MORPH	
Maior     	maior     	ADJ	ADJ	nsubj     	Xxxxx	True	True	adjective 	Gender=Fem|Number=Sing	
medalhista	medalhista	ADJ	ADJ	amod      	xxxx	True	False	adjective 	Gender=Fem|Number=Sing	
da        	de o      	ADP	ADP	case      	xx	True	True	adposition	Definite=Def|Gender=Fem|Number=Sing|PronType=Art	
história  	história  	NOUN	NOUN	nmod      	xxxx	True	False	noun      	Gender=Fem|Number=Sing	
do        	de o      	ADP	ADP	case      	xx	True	True	adposition	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	
Brasil    	Brasil    	PROPN	PROPN	nmod      	Xxxxx	True	False	proper noun	Gender=Masc|Number=Sing	
,         	,         	PUNCT	PUNCT	punct     	,	False	False	punctuation		
Rebeca    	Rebeca    	PROPN	PROPN	appos     	Xxxxx	True	False	proper noun	Gender=Fem|Number=Sing	
Andrade   	Andrade   	PROPN	PROPN	flat:name 	Xxxxx	True	False	proper noun	Number=Sing	
foi       	ser       	AUX	AUX	cop       	xxx	True	True	auxi

In [6]:
doc = nlp(text)
displacy.render(doc, style="ent", jupyter=True)

In [7]:
doc.ents

(Brasil, Rebeca Andrade, Olimpíadas de Paris)

In [8]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Brasil LOC
Rebeca Andrade PER
Olimpíadas de Paris MISC


In [9]:
doc = nlp('Maior medalhista da história do Brasil.')
displacy.render(doc, style="dep", jupyter=True)

##### Processando .txt

In [10]:
#load text file
with open('../data/doc.txt', encoding='utf-8') as f:
    txt = f.read()
    print(txt)

Rebeca foi a atleta brasileira que mais faturou nas Olimpíadas

Rebeca Andrade, a maior medalhista da história do Brasil, foi a atleta que mais recebeu premiação pelo desempenho nas Olimpíadas de Paris
Postado em 11 de agosto de 2024 por Matheus Santana

Rebeca Andrade, a maior medalhista da história do Brasil, foi a atleta que mais recebeu premiação pelo desempenho nas Olimpíadas de Paris. Com quatro medalhas conquistadas (um ouro, duas pratas e um bronze), a ginasta embolsou R$ 826 mil do Comitê Olímpico do Brasil (COB). Além de ser a brasileira com o maior número de medalhas no evento, Rebeca se destacou como a atleta mais premiada do país.

Outras atletas que também brilharam nas Olimpíadas, como Bia Souza, que garantiu uma medalha de bronze por equipes no judô, recebeu R$ 392 mil. Já as campeãs de vôlei de praia, Ana Patrícia e Duda, faturaram R$ 350 mil cada uma.

No total, o Brasil conquistou 20 medalhas em Paris. O COB definiu uma premiação diferenciada para esportes individuai

In [11]:
doc = nlp(txt)
per_counter = Counter()
loc_counter = Counter()
per_ents = [ent.text.lower() for ent in doc.ents if ent.label_ == "PER"]
loc_ents = [ent.text.lower() for ent in doc.ents if ent.label_ == "LOC"]
per_counter.update(per_ents)
loc_counter.update(loc_ents)

In [12]:
per_counter

Counter({'rebeca': 2,
         'matheus santana\n\nrebeca andrade': 1,
         'bia souza': 1,
         'ana patrícia': 1,
         'duda': 1,
         'r$': 1})

In [13]:
loc_counter

Counter({'brasil': 3, 'cob': 1, 'paris': 1, 'vale': 1})

##### Processando .pdf

In [14]:
doe = PdfReader('../data/doc.pdf')
print(len(doe.pages))

1


In [15]:
per_counter = Counter()
loc_counter = Counter()

for page in doe.pages:
    text = page.extract_text()
    doc = nlp(text)
    per_ents = [ent.text.lower() for ent in doc.ents if ent.label_ == "PER"]
    loc_ents = [ent.text.lower() for ent in doc.ents if ent.label_ == "LOC"]
    per_counter.update(per_ents)
    loc_counter.update(loc_ents)

In [16]:
per_counter

Counter({'rebeca': 2,
         'matheus santana\nrebeca andrade': 1,
         'bia souza': 1,
         'ana patrícia': 1,
         'duda': 1,
         'r$': 1})

In [17]:
loc_counter

Counter({'brasil': 3, 'cob': 1, 'paris': 1, 'r$': 1, 'vale': 1})

##### Processando .png

In [18]:
from PIL import Image

import pytesseract
img = Image.open('../data/doc.png')
text = pytesseract.image_to_string(img)

In [19]:
print(text)

Rebeca foi a atleta brasileira que mais faturou nas Olimpiadas
‘Matheus Santana

Rebeca Andrade, a maior medalhista da histéria do Brasil, foi a atleta que mais recebeu premiagio pelo

desempenho nas Olimpfadas de Paris,
Postado em 11 de agosto de 2024 por Matheus Santana

Rebeca Andrade, a maior medalhista da histéria do Brasil, foi a atleta que mais recebeu premiagio pelo
desempenho nas Olimpiadas de Paris. Com quatro medalhas conquistadas (um ouro, duas pratas e um bronze), a
ginasta embolsou RS 826 mil do Comité Olimpico do Brasil (COB). Além de ser a brasileira com 0 maior

nimero de medalhas no evento, Rebeca se destacou como a atleta mais premiada do pais.

Outras atletas que também brilharam nas Olimpiadas, como Bia Souza, que garantiu uma medalha de bronze

 

por equipes no jud6, recebeu R$ 392 mil. Jé as campetis de vélei de praia, Ana Patricia e Duda, faturaram RS

 

350 mil cada uma.

   

No total, o Brasil conquistou 20 medalhas em Paris. 0 COB definiu uma premiacdo dif

In [20]:
doc = nlp(text)
per_counter = Counter()
loc_counter = Counter()
per_ents = [ent.text.lower() for ent in doc.ents if ent.label_ == "PER"]
loc_ents = [ent.text.lower() for ent in doc.ents if ent.label_ == "LOC"]
per_counter.update(per_ents)
loc_counter.update(loc_ents)

In [21]:
per_counter

Counter({'rebeca': 2,
         'matheus santana\n\nrebeca andrade': 2,
         'bia souza': 1,
         'ana patricia': 1,
         'duda': 1})

In [22]:
loc_counter

Counter({'rs': 4,
         'brasil': 3,
         'olimpiadas': 2,
         'postado': 1,
         'cob': 1,
         'paris': 1,
         'rs\n\n \n\nvale': 1})